Jupyter Notebook to train a residual autoencoder for peak signal reconstruction.

Specifically designed for *Google Colab*.

Author: **Fabian Jakob**

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import pickle
import random as rnd
from tqdm import tqdm

import torch
from torch import nn, optim, flatten, sigmoid
from torch.utils.data import TensorDataset, random_split, DataLoader
from torchsummary import summary

from google.colab import files


For GPU support:

In [ ]:
if not torch.cuda.is_available():
  print("Set google colab runtime to GPU!")
else:
  device = "cuda"
  print("Using " + device)
  device = torch.device(device)


Define Neural Network Encoder/Decoder Structures:

In [ ]:
class ResidualBlock(nn.Module):
    """ Residual Block, consisting of two channel and size invariant convolutions plus shortcut. """
    def __init__(self, channel_num, kernel_size, mode='encode'):
        super().__init__()

        padding = int((kernel_size-1)/2)
        if mode=='encode':
            conv1 = nn.Conv1d(channel_num, channel_num, kernel_size, padding=padding)
            conv2 = nn.Conv1d(channel_num, channel_num, kernel_size, padding=padding)
        elif mode=='decode':
            conv1 = nn.ConvTranspose1d(channel_num, channel_num, kernel_size, padding=padding)
            conv2 = nn.ConvTranspose1d(channel_num, channel_num, kernel_size, padding=padding)
        
        self.conv_block1 = nn.Sequential(conv1, nn.BatchNorm1d(channel_num), nn.ReLU()) 
        self.conv_block2 = nn.Sequential(conv2, nn.BatchNorm1d(channel_num), nn.ReLU()) 
        self.activation = nn.ReLU()

    def forward(self, x):
        residual = x
        x = self.conv_block1(x)
        x = self.conv_block2(x)
        x = x + residual
        return x


class ResidualStack(nn.Module):
    """ Residual Unit consisting of multiple residual Blocks. """
    def __init__(self, channel_in, channel_out, kernel_size, mode='encode'):
        super().__init__()

        # if necessary, perform dimension transformation to output channel
        if channel_in == channel_out:
            self.dimension_mismatch = False
        else:
            self.dimension_mismatch = True
            if mode=='encode':
                self.dim_transition = nn.Conv1d(channel_in, channel_out, kernel_size=1)
            elif mode=='decode':
                self.dim_transition = nn.ConvTranspose1d(channel_in, channel_out, kernel_size=1)

        self.res_blk1 = ResidualBlock(channel_out, kernel_size, mode)
        self.res_blk2 = ResidualBlock(channel_out, kernel_size, mode)

    def forward(self, x):
        x = self.dim_transition(x) if self.dimension_mismatch else x
        x = self.res_blk1(x)
        x = self.res_blk2(x)
        return(x)


class Encoder(nn.Module):
    """ Encoder neural network consisting of 4 convolutional layers. """
    def __init__(self, latent_dim):
        super().__init__()
        self.activation = nn.ReLU()
        self.pool   = nn.MaxPool1d(3)
        self.conv1  = nn.Conv1d(1, 16, 7)
        self.conv2  = nn.Conv1d(16, 32, 6)
        self.conv3  = nn.Conv1d(32, 64, 6)
        self.conv4  = nn.Conv1d(64, 128, 5)
        self.dense1 = nn.Linear(1280, latent_dim)
        self.bn1 = nn.BatchNorm1d(16)
        self.bn2 = nn.BatchNorm1d(32)
        self.bn3 = nn.BatchNorm1d(64)
        self.bn4 = nn.BatchNorm1d(128)
        self.drop = nn.Dropout(0.3)
        
    def forward(self, input):
        out = self.pool(self.activation(self.conv1(input)))
        out = self.bn1(out)
        out = self.drop(out)
        out = self.pool(self.activation(self.conv2(out)))
        out = self.bn2(out)
        out = self.drop(out)
        out = self.pool(self.activation(self.conv3(out)))
        out = self.bn3(out)
        out = self.drop(out)
        out = self.pool(self.activation(self.conv4(out)))
        out = self.bn4(out)
        out = self.drop(out)
        out = flatten(out, 1)
        out = self.dense1(out)
        return out  


class ResidualEncoder(nn.Module):
    """ Encoder neural network consisting of 4 residual unis. """
    def __init__(self, latent_dim):
        super().__init__()
        self.pool   = nn.MaxPool1d(3)
        self.drop = nn.Dropout(0.3)
        self.resUnit1 = ResidualStack(1, 16, kernel_size=3)
        self.resUnit2 = ResidualStack(16, 32, kernel_size=3)
        self.resUnit3 = ResidualStack(32, 64, kernel_size=3)
        self.resUnit4 = ResidualStack(64, 128, kernel_size=3)
        self.dense    = nn.Linear(128*12, latent_dim)

    def forward(self, x):
        x = self.pool(self.resUnit1(x))       
        x = self.pool(self.resUnit2(x))
        x = self.pool(self.resUnit3(x))
        x = self.pool(self.resUnit4(x))
        x = flatten(x, 1)
        x = self.dense(x)
        return x
    

class Decoder(nn.Module):
    """ Decoder neural network consisting of 4 inverted convolutional layers. """
    def __init__(self, latent_dim):
        super().__init__()
        self.activation = nn.ReLU()
        self.dense1   = nn.Linear(latent_dim, 128*36)
        self.conv1T   = nn.ConvTranspose1d(128, 64, 7, stride=3)
        self.conv2T   = nn.ConvTranspose1d(64, 32, 6, stride=3)
        self.conv3T   = nn.ConvTranspose1d(32, 16, 6, stride=3)
        self.conv4T   = nn.ConvTranspose1d(16, 1, 5)
        self.bn1 = nn.BatchNorm1d(128)
        self.bn2 = nn.BatchNorm1d(64)
        self.bn3 = nn.BatchNorm1d(32)
        self.bn4 = nn.BatchNorm1d(16)
            
    def forward(self, input):
        out = self.activation(self.dense1(input))
        out = out.view(-1, 128, 36)
        out = self.bn1(out)
        out = self.activation(self.conv1T(out))
        out = self.bn2(out)
        out = self.activation(self.conv2T(out))
        out = self.bn3(out)
        out = self.activation(self.conv3T(out))
        out = self.bn4(out)
        out = self.conv4T(out)
        return out  

class DeepDecoder(nn.Module):
    """ Decoder neural network consisting of 5 inverted convolutional layers. """
    def __init__(self, latent_dim):
        super().__init__()
        self.activation = nn.ReLU()
        self.dense1   = nn.Linear(latent_dim, 128*36)
        self.conv1T   = nn.ConvTranspose1d(128, 64, 7, stride=3)
        self.conv2T   = nn.ConvTranspose1d(64, 32, 6, stride=3)
        self.conv3T   = nn.ConvTranspose1d(32, 16, 6, stride=3)
        self.conv4T   = nn.ConvTranspose1d(16, 8, 3, padding=1)
        self.conv5T   = nn.ConvTranspose1d(8, 4, 3, padding=1)
        self.conv6T   = nn.ConvTranspose1d(4, 1, 5)
        self.bn1 = nn.BatchNorm1d(128)
        self.bn2 = nn.BatchNorm1d(64)
        self.bn3 = nn.BatchNorm1d(32)
        self.bn4 = nn.BatchNorm1d(16)
        self.bn5 = nn.BatchNorm1d(8)
        self.bn6 = nn.BatchNorm1d(4)
        self.epsilon = 1e-12
            
    def forward(self, input):
        out = self.activation(self.dense1(input))
        out = out.view(-1, 128, 36)
        out = self.bn1(out)
        out = self.activation(self.conv1T(out))
        out = self.bn2(out)
        out = self.activation(self.conv2T(out))
        out = self.bn3(out)
        out = self.activation(self.conv3T(out))
        out = self.bn4(out)
        out = self.activation(self.conv4T(out))
        out = self.bn5(out)
        out = self.activation(self.conv5T(out))
        out = self.bn6(out)
        out = (1+self.epsilon)*sigmoid(self.conv6T(out)) - self.epsilon/2
        return out  


class Autoencoder(nn.Module):
    """ Autoencoder for dimensionality reduction. """
    def __init__(self, latent_dim=12):
        super(Autoencoder, self).__init__()
        self.encoder = ResidualEncoder(latent_dim)
        self.decoder = DeepDecoder(latent_dim)
        
    def forward(self, input):
        encoded = self.encoder(input)
        decoded = self.decoder(encoded)
        return decoded

Data loading and preparation:

In [ ]:
def loadDataSet(filename):
    with open(filename, 'rb') as f:
        X = pickle.load(f)
    return X

def createDataset(X, device="cpu", batch_size=128, test_split=0.2):
    # split to test / train
    n_data = len(X)
    n_test = int(test_split*n_data)
    n_train = n_data - n_test

    X = torch.tensor(X, device=device)
    dataSet = TensorDataset(X.float())

    train_dataset, test_dataset = random_split(dataSet, [n_train, n_test])
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
    return train_loader, test_loader

Setup Printing functions:

In [ ]:
def printTrainingHistory(latent_dim, schedule_epochs=False):
    file = "train_history_{0}.pickle".format(latent_dim)
    if os.path.isfile(file):
        with open(file, 'rb') as f:
            hist, _ = pickle.load(f)
    else: 
        print("No history!")
        return
    train_loss = [epoch[0] for epoch in hist]
    test_loss = [epoch[1] for epoch in hist]
    print("Training loss: {}, Test loss: {}".format(train_loss[-1], test_loss[-1]))
    fig = plt.figure()
    ax = fig.add_subplot(1,1,1)
    ax.plot(train_loss, label='Train loss')
    ax.grid()
    ax.plot(test_loss, label='Test loss')
    ax.legend()
    ax.set_ylim((0,35/10000))
    plt.show()

def printSamplePredict(test_loader, autoencoder, device, sample=1):
    n_subplots = 1
    plt.figure(figsize=(16,4))
    with torch.no_grad():
        jdx=0
        for batch in test_loader:
            jdx+=1
            batch_in = torch.reshape(batch[0].to(device), (batch[0].size()[0], 1, 1024))
            batch_enc = autoencoder.encoder(batch_in)
            batch_out = autoencoder(batch_in)
            if jdx==sample:
                for idx in range(n_subplots):
                    r = rnd.randint(0, len(batch))
                    plt.subplot(n_subplots,2,idx+1)
                    plt.plot(np.squeeze(batch_in[r].cpu().detach().numpy()), label='Original')
                    plt.plot(np.squeeze(batch_out[r].cpu().detach().numpy()), '--', label='Reconstructed')
                    plt.legend()
                    plt.subplot(n_subplots,2,idx+2)
                    plt.plot(np.squeeze(batch_enc[r].cpu().detach().numpy()))
                break;
    plt.show()


Train the Autoencoder:



In [ ]:
# load data
print("----------- Loading data -----------")
X = loadDataSet('data_w30_unlabeled_normalized_augmented.pickle')
print("Done")

In [ ]:
# hyperparameters
epochs = 200
batch_size = 512
learning_rate = 0.01
latent_dim = 24

# dataset loader
train_loader, test_loader = createDataset(X, device, batch_size, test_split=0.2)

# instanciate model
print("---------- Creating model ----------")
autoencoder = Autoencoder(latent_dim).to(device)
summary(autoencoder, (1,1024))
optimizer = optim.Adam(autoencoder.parameters(), lr=learning_rate)
loss_function = nn.MSELoss()

# learning rate schedule variables
count = 0
loss_storage = []
schedule_epochs = []
schedule_horizon = 30
train_stagnation = 0.5/10000

print("---------- Start training ----------")
for epoch in range(epochs):

    # learning rate schedule:
    count += 1
    if count > schedule_horizon:

        # track variation in training loss
        recent_loss = [train_loss for train_loss, test_loss in loss_storage[-schedule_horizon:]]
        loss_change = np.std(recent_loss)

        if loss_change < train_stagnation:
            learning_rate = learning_rate / 2
            optimizer = optim.Adam(autoencoder.parameters(), lr=learning_rate)

            print(f"-------- Learning rate schedule performed from {2*learning_rate} to {learning_rate} ----------")
            schedule_horizon += 5
            schedule_epochs.append(epoch)
            count = 0
            train_stagnation = train_stagnation / 2

    loss = 0
    for batch_original in train_loader:
        # preprocess batch as model input
        batch_original = torch.reshape(batch_original[0].to(device), (batch_original[0].size()[0], 1, 1024)).to(device)
        
        # reset gradient
        optimizer.zero_grad() 
        
        # compute loss
        batch_decoded = autoencoder(batch_original)
        train_loss = loss_function(batch_original, batch_decoded)
        
        # compute gradient + perform gradient decent
        train_loss.backward()
        optimizer.step()
        
        loss += train_loss.item()

    # epoch training loss
    loss = loss / len(train_loader)
    
    # compute test loss
    with torch.no_grad():
        test_loss = 0
        for test_batch in test_loader:
            test_batch = torch.reshape(test_batch[0].to(device), (test_batch[0].size()[0], 1, 1024))

            test_batch_decoded = autoencoder(test_batch)
            test_loss += loss_function(test_batch, test_batch_decoded).item()

        test_loss = test_loss / len(test_loader)
    
    loss_storage.append((loss, test_loss))
        
    print("epoch : {}/{}, loss = {:.2f}, test loss = {:.2f}".format(epoch + 1, epochs, loss*10000, test_loss*10000)) 


print("--------- Printing results ----------")

for idx in range(8):
    printSamplePredict(test_loader, autoencoder, device, sample=idx+1)


print("---------- Saving results ----------")

filename = f"autoencoder{latent_dim}.pth"

torch.save(autoencoder.state_dict(), filename)
files.download(filename)

with open(f"train_history_{latent_dim}.pickle", "wb") as fp:
    pickle.dump((loss_storage, schedule_epochs), fp)
files.download(f"train_history_{latent_dim}.pickle")

printTrainingHistory(latent_dim)

print("Done.")

In [ ]:
printTrainingHistory(latent_dim)